In [2]:
import cv2
import os
import torch
import json
import numpy as np
from tqdm.notebook import tqdm
from copy import deepcopy

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from mmcv.transforms import Compose
from mmdet.utils import get_test_pipeline_cfg

def read_json(json_path):
    with open(json_path) as f:
        data = json.load(f)
    return data

def read_txt(txt_path):
    with open(txt_path) as f:
        data = f.readlines()
    data = [x.strip() for x in data]
    return data

def preprocess(test_pipeline, image):
    if isinstance(image, np.ndarray):
        # Calling this method across libraries will result
        # in module unregistered error if not prefixed with mmdet.
        test_pipeline[0].type = 'mmdet.LoadImageFromNDArray'
    test_pipeline = Compose(test_pipeline)
    return test_pipeline(dict(img=image))

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, annotations_json_path, transform=None):
        self.transform = transform
        self.images_dir = images_dir
        self.annotations_json = read_json(annotations_json_path)


    def __len__(self):
        return len(self.annotations_json['images'])

    def __getitem__(self, idx):
        image_dict = self.annotations_json['images'][idx]
        image_path = os.path.join(self.images_dir, image_dict['file_name'])
        image_id = image_dict['id']

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            transformed_images = self.transform(image)
        else:
            transformed_images = image

        return image_id, image_path, transformed_images


# calibrationDataloader = DataLoader(calibrationDataset, batch_size=32, shuffle=True)

In [3]:
import torch
from mmdet.apis import DetInferencer

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize([640, 640]),  # Resize
])

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CONFIG_PATH = '/teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco.py'
WEIGHTS_PATH = '/teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'
EVAL_DATASET_SIZE = 5000
CALIBRATION_DATASET_SIZE = 1000
BATCH_SIZE = 64

ROOT_DATASET_DIR = '/teamspace/studios/this_studio/COCO'
IMAGES_DIR = os.path.join(ROOT_DATASET_DIR, 'images')
ANNOTATIONS_JSON_PATH = os.path.join(ROOT_DATASET_DIR, 'annotations/instances_val2017.json')
# ANNOTATIONS_JSON_PATH = "/home/shayaan/Desktop/aimet/my_mmdet/temp.json"

model = DetInferencer(model=CONFIG_PATH, weights=WEIGHTS_PATH, device=DEVICE)
evalDataset = CustomImageDataset(images_dir=IMAGES_DIR, annotations_json_path=ANNOTATIONS_JSON_PATH, transform=transform)
eval_data_loader = DataLoader(evalDataset, batch_size=BATCH_SIZE)
calibration_images = read_txt('/teamspace/studios/this_studio/aimet/Examples/torch/quantization/calibration_image_ids.txt')
calibration_data_loader = DataLoader(calibration_images, batch_size=BATCH_SIZE)

DEVICE

[2024-09-09 13:07:03,421] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible
Loads checkpoint by local backend from path: /teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std

09/09 13:07:06 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


device(type='cuda', index=0)

In [4]:
from collections import OrderedDict
from copy import deepcopy

m = deepcopy(model.model)

def is_leaf(module): 
    return len(module._modules) == 0

def replace_bn(m):

    if is_leaf(m):
        return 

    for _, child in m.named_children(): 
        
        if "bn" in child._modules.keys():
            bn = child._modules.get("bn")
            bn_params = deepcopy(bn._parameters)
            bn_buffers = deepcopy(bn._buffers)
            new_bn = torch.nn.BatchNorm2d(bn.num_features, eps=bn.eps, momentum=bn.momentum, affine=bn.affine, track_running_stats=bn.track_running_stats)
            new_bn._parameters["weight"].data = bn_params["weight"].data
            new_bn._parameters["bias"].data = bn_params["bias"].data
            new_bn._buffers["running_mean"].data = bn_buffers["running_mean"].data
            new_bn._buffers["running_var"].data = bn_buffers["running_var"].data
            new_bn._buffers["num_batches_tracked"].data = bn_buffers["num_batches_tracked"].data
            child._modules["bn"] = new_bn
            
        replace_bn(child)


In [5]:
model = deepcopy(model.model)
replace_bn(model)

In [6]:
from aimet_torch.model_preparer import prepare_model

model = prepare_model(model)

2024-09-09 13:07:08,580 - root - INFO - AIMET
2024-09-09 13:07:08,764 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.blocks.0.module_add} 
2024-09-09 13:07:08,766 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.module_cat} 
2024-09-09 13:07:08,767 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.attention.module_mul} 
2024-09-09 13:07:08,768 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage2.1.blocks.0.module_add_1} 
2024-09-09 13:07:08,769 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage2.1.module_cat_1} 
2024-09-09 13:07:08,770 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage2.1.attention.module_mul_1} 
2024-09-09 13:07:08,772 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage3.1.blocks.0.modu

In [7]:
def read_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)

In [8]:
for k in dict(model.named_modules()).keys():
    print(k)


backbone
backbone.stem
backbone.stem.0
backbone.stem.0.conv
backbone.stem.0.bn
backbone.stem.0.activate
backbone.stem.0.activate.sigmoid
backbone.stem.0.activate.mul
backbone.stem.1
backbone.stem.1.conv
backbone.stem.1.bn
backbone.stem.1.activate
backbone.stem.1.activate.sigmoid
backbone.stem.1.activate.mul
backbone.stem.2
backbone.stem.2.conv
backbone.stem.2.bn
backbone.stem.2.activate
backbone.stem.2.activate.sigmoid
backbone.stem.2.activate.mul
backbone.stage1
backbone.stage1.0
backbone.stage1.0.conv
backbone.stage1.0.bn
backbone.stage1.0.activate
backbone.stage1.0.activate.sigmoid
backbone.stage1.0.activate.mul
backbone.stage1.1
backbone.stage1.1.short_conv
backbone.stage1.1.short_conv.conv
backbone.stage1.1.short_conv.bn
backbone.stage1.1.short_conv.activate
backbone.stage1.1.short_conv.activate.sigmoid
backbone.stage1.1.short_conv.activate.mul
backbone.stage1.1.main_conv
backbone.stage1.1.main_conv.conv
backbone.stage1.1.main_conv.bn
backbone.stage1.1.main_conv.activate
backbone

In [9]:
modules_to_change = ["backbone.stage2.1.blocks.0.conv2.depthwise_conv.conv", "backbone.stage1.1.blocks.0.conv2.depthwise_conv.conv", "backbone.stage2.1.blocks.0.conv2.pointwise_conv.conv", "backbone.stage3.1.blocks.0.conv2.depthwise_conv.conv", "backbone.stage4.2.blocks.0.conv2.depthwise_conv.conv", "neck.top_down_blocks.0.blocks.0.conv2.depthwise_conv.conv", "neck.top_down_blocks.1.blocks.0.conv2.depthwise_conv.conv", "neck.bottom_up_blocks.0.blocks.0.conv2.depthwise_conv.conv", "neck.bottom_up_blocks.1.blocks.0.conv2.depthwise_conv.conv"]


In [13]:
from glob import glob

DIR = "./quant_analyzer_int16"
MODULE = modules_to_change[0]

stats_path = os.path.join(DIR, "activations_pdf", MODULE)
stats_path = glob(f"{stats_path}*.json")

for s in stats_path:
    print(s)
# os.path.exists(stats_path)

In [11]:
obj = read_json(stats_path[0])

pdfs = obj["pdfs"]
encoding_min = obj["encoding_min"]
encoding_max = obj["encoding_max"]

encoding_min, encoding_max

(-14.177273750305176, 14.313347816467285)

In [12]:
from pprint import pprint

pdfs = {float(k):v for k, v in pdfs.items()}
pdfs = {k:v for k, v in sorted(pdfs.items(), key=lambda x: x[0], reverse=True)[::-1]}

pprint(pdfs)

{0.0: [-38.340370178222656,
       -38.186466217041016,
       -38.03255844116211,
       -37.87865447998047,
       -37.72474670410156,
       -37.57084274291992,
       -37.416934967041016,
       -37.263031005859375,
       -37.10912322998047,
       -36.95521926879883,
       -36.80131149291992,
       -36.64740753173828,
       -36.493499755859375,
       -36.339595794677734,
       -36.18568801879883,
       -36.03178405761719,
       -35.87787628173828,
       -35.72397232055664,
       -35.570068359375,
       -35.416160583496094,
       -35.26225662231445,
       -35.10834884643555,
       -34.954444885253906,
       -34.800537109375,
       -34.64663314819336,
       -34.49272537231445,
       -34.33882141113281,
       -34.184913635253906,
       -34.031009674072266,
       -33.87710189819336,
       -33.72319793701172,
       -33.56929016113281,
       -33.41538619995117,
       -33.26148223876953,
       -33.107574462890625,
       -32.953670501708984,
       -32.799762725

In [87]:
vals = [item for sublist in list(pdfs.values()) for item in sublist]
min(vals), max(vals)

(-77.95208740234375, 85.07522583007812)

In [13]:
pdfs.pop(0.0)
vals = [item for sublist in list(pdfs.values()) for item in sublist]
min(vals), max(vals)

(-28.820568084716797, 34.986595153808594)

In [14]:
pprint(pdfs)

{1.5894571940104166e-09: [-28.820568084716797,
                          -26.587318420410156,
                          -26.26828384399414,
                          -25.311176300048828],
 1.5894571940104168e-09: [-24.992141723632812, -24.67310333251953],
 1.589457194010417e-09: [31.477203369140625,
                         33.71044921875,
                         34.986595153808594],
 3.178914388020833e-09: [31.158164978027344, 31.796234130859375],
 4.768371582031251e-09: [30.520095825195312,
                         32.115272521972656,
                         32.43431091308594],
 6.357828776041666e-09: [-21.482746124267578],
 6.357828776041667e-09: [-23.71599578857422, -23.396961212158203],
 6.357828776041668e-09: [-24.0350341796875],
 7.947285970052083e-09: [-24.354068756103516,
                         -23.077926635742188,
                         -22.758888244628906,
                         -22.120819091796875],
 9.5367431640625e-09: [30.839126586914062],
 1.2715657552083334e-08